In [2]:
from google.auth import default
credentials, project = default()

In [3]:
%load_ext autoreload
%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()
from tqdm import tqdm

from src.utils import (load_json, get_first_n_items, save_json, save_text_file, load_text_file)  # noqa: E402
from src.story import upload_story_image, prepare_dialogue_with_wiktionary
from src.translation import review_story_dialogue_translations
from src.gcs_storage import check_blob_exists, read_from_gcs, upload_to_gcs, get_story_translated_dialogue_path, get_story_dialogue_path
from src.config_loader import config
config.TARGET_LANGUAGE_NAME

Default FFmpeg path added to system PATH: C:\Program Files\ffmpeg-7.0-essentials_build\bin


'Swedish'

# Story Data

* based around a story_name which is part of a collection (LM1000 > story_the_birthday_party)
* translated dialogue (text file)
    * comes from google translate
    * added to with Wiktionary links for each utternace ('wiktionary_links')
* audio file for each utterance of Sam and Alex
* 1 x fast audio file for each story part, introduction etc
* an image for each story part
* challenges for each story
    * from a single challenges.json file for each story
    * the customisation is when the challenges.html file gets created as the language name is added into the prompt

## Storage overview

We want to save the english story dialogue as dialogue.json

Then translations by language_name, then each phrase as an audio clip. We might want higher quality audio for the stories e.g. chirp3 voice

```raw
collections/LM1000/stories/story_murder_mystery/dialogue.json
collections/LM1000/stories/story_murder_mystery/dialogue/french/translated_dialogue.json
collections/LM1000/stories/story_murder_mystery/images/introduction.png etc
collections/LM1000/stories/story_murder_mystery/audio/french/introduction/part_0_sam.mp3
collections/LM1000/stories/story_murder_mystery/audio/french/introduction/part_1_alex.mp3
collections/LM1000/stories/story_murder_mystery/audio/french/introduction/fast/part_1_alex.mp3```


This allows us to quickly retrieve a phrase based on the bucket name and the phrase key, as well as modify individual phrases for later correction

In [3]:
config._load_config()
language_name = config.TARGET_LANGUAGE_NAME.lower()
print(language_name)
config.get_voice_models(enum_type="stories")

swedish


(VoiceInfo(name='en-GB-Chirp3-HD-Algenib', provider=<VoiceProvider.GOOGLE: 'google'>, voice_id='en-GB-Chirp3-HD-Algenib', language_code='en-GB'),
 VoiceInfo(name='aSLKtNoVBZlxQEMsnGL2', provider=<VoiceProvider.ELEVENLABS: 'elevenlabs'>, voice_id='aSLKtNoVBZlxQEMsnGL2', language_code='sv-SE'),
 VoiceInfo(name='7UMEOkIJdI4hjmR2SWNq', provider=<VoiceProvider.ELEVENLABS: 'elevenlabs'>, voice_id='7UMEOkIJdI4hjmR2SWNq', language_code='sv-SE'))

In [4]:
from src.gcs_storage import get_stories_from_collection

all_stories = get_stories_from_collection(collection="LM1000")

In [5]:
all_stories

['story_sunset_wedding_blues',
 'story_better_than_a_movie',
 'story_unexpected_wedding_guests',
 'story_midnight_garden_mystery',
 'story_workplace_stress_vacation',
 'story_the_marathon',
 'story_teaching_music',
 'story_job_interview_gone_wrong',
 'story_winter_in_sweden',
 'story_a_fishing_trip',
 'story_team_spirit',
 'story_brussels_blend',
 'story_rainy_football_match',
 'story_camping_trip_gone_awry',
 'story_unexpected_career_change',
 'story_a_missed_stop',
 'story_road_trip',
 'story_forgetful_diver',
 'story_the_birthday_party',
 'story_the_power_cut']

## Loop all stories (text)

Translate and re-upload

In [ ]:
# loop stories and translate
from src.gcs_storage import check_blob_exists, get_story_dialogue_path, get_story_translated_dialogue_path
from src.dialogue_generation import translate_and_upload_dialogue
from src.translation import review_story_dialogue_translations

for story_name in all_stories:
    # get the dialogue
    story_file_path = get_story_dialogue_path(story_name, collection="LM1000")
    translated_file_path = get_story_translated_dialogue_path(story_name, collection="LM1000")
    if check_blob_exists(config.GCS_PRIVATE_BUCKET, translated_file_path):
        print(f"{story_name} already translated")
        #continue
    story_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, story_file_path)
    translate_and_upload_dialogue(story_dialogue, story_name, collection="LM1000")

## Review translations with LLM

In [11]:
for story_name in all_stories:
    print(f"processing {story_name}")
    # get the dialogue
    translated_file_path = get_story_translated_dialogue_path(story_name, collection="LM1000")
    translated_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, translated_file_path)
    reviewed_dialogue = review_story_dialogue_translations(translated_dialogue)
    upload_to_gcs(obj=reviewed_dialogue, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=translated_file_path)

processing story_sunset_wedding_blues
processing story_better_than_a_movie
processing story_unexpected_wedding_guests
processing story_midnight_garden_mystery
processing story_workplace_stress_vacation
processing story_the_marathon
processing story_teaching_music
processing story_job_interview_gone_wrong
processing story_winter_in_sweden
processing story_a_fishing_trip
processing story_team_spirit
processing story_brussels_blend
processing story_rainy_football_match
processing story_camping_trip_gone_awry
processing story_unexpected_career_change
processing story_a_missed_stop
processing story_road_trip
processing story_forgetful_diver
processing story_the_birthday_party
processing story_the_power_cut


## Add wiktionary links to each story utterance

In [12]:
for story_name in all_stories:
    print(f"processing {story_name}")
    # get the dialogue
    translated_file_path = get_story_translated_dialogue_path(story_name, collection="LM1000")
    if not check_blob_exists(config.GCS_PRIVATE_BUCKET, translated_file_path):
        print(f"{story_name} not yet translated")
        continue
    translated_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, translated_file_path)
    translated_dialogue_with_links = prepare_dialogue_with_wiktionary(translated_dialogue)
    # now re-upload it with embedded witkionary_links
    uploaded = upload_to_gcs(obj=translated_dialogue_with_links, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=translated_file_path)
    print(f"uploaded {story_name} : {uploaded}")

processing story_sunset_wedding_blues
Got word link cache of size 2437 from GCS


Getting dialogue links for story_parts: 100%|██████████| 3/3 [00:08<00:00,  2.96s/it]


uploaded story_sunset_wedding_blues : gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_sunset_wedding_blues/translated_dialogue.json
processing story_better_than_a_movie
Got word link cache of size 2450 from GCS


Getting dialogue links for story_parts: 100%|██████████| 3/3 [00:08<00:00,  2.73s/it]


uploaded story_better_than_a_movie : gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_better_than_a_movie/translated_dialogue.json
processing story_unexpected_wedding_guests
Got word link cache of size 2460 from GCS


...

uploaded story_forgetful_diver : gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_forgetful_diver/translated_dialogue.json
processing story_the_birthday_party
Got word link cache of size 2668 from GCS


Getting dialogue links for story_parts: 100%|██████████| 3/3 [00:05<00:00,  1.78s/it]


uploaded story_the_birthday_party : gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_the_birthday_party/translated_dialogue.json
processing story_the_power_cut
Got word link cache of size 2679 from GCS


Getting dialogue links for story_parts: 100%|██████████| 3/3 [00:08<00:00,  2.69s/it]


uploaded story_the_power_cut : gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_the_power_cut/translated_dialogue.json


Generate audio and upload

### Generate audio and upload

### Loop through stories to generate audio

In [1]:
all_stories

NameError: name 'all_stories' is not defined

In [13]:
from src.audio_generation import generate_dialogue_audio_and_upload
for story_name in all_stories:
    # get the dialogue
    translated_file_path = get_story_translated_dialogue_path(story_name, collection="LM1000")

    translated_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, translated_file_path)
    generate_dialogue_audio_and_upload(translated_dialogue, story_name, overwrite=True)

Processing story parts:   0%|          | 0/3 [00:00<?, ?it/s]

Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_sunset_wedding_blues/audio/introduction/part_0_alex.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_sunset_wedding_blues/audio/introduction/part_1_sam.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_sunset_wedding_blues/audio/introduction/part_2_alex.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_sunset_wedding_blues/audio/introduction/part_3_sam.mp3


...

Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_job_interview_gone_wrong/audio/development/part_7_sam.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_job_interview_gone_wrong/audio/resolution/part_0_alex.mp3


Generated and uploaded: gs://audio-language-trainer-private-content/collections/LM1000/swedish/stories/story_job_interview_gone_wrong/audio/resolution/part_1_sam.mp3


Processing story parts:  67%|██████▋   | 2/3 [02:28<01:14, 74.43s/it]


Exception: ElevenLabs speech synthesis error: API request failed with status code 401: {"detail":{"status":"quota_exceeded","message":"This request exceeds your quota of 32184. You have 11 credits remaining, while 81 credits are required for this request."}}

### Fast Audio 
1 file for each story part

In [ ]:
from src.audio_generation import generate_and_upload_fast_audio

for story_name in tqdm(all_stories):
    # uploads 1 fast mp3 file for each story part. 
    generate_and_upload_fast_audio(story_name, collection="LM1000", overwrite=True)

## Upload images
For when we have them stored locally

In [ ]:
for story_name in all_stories:
    notebook_dir = Path().absolute()  # This gives src/notebooks
    story_dir = notebook_dir.parent / "outputs" / "stories"
    # get the dialogue
    story_file_path = get_story_dialogue_path(story_name, collection="LM1000")
    story_dialogue = read_from_gcs(config.GCS_PRIVATE_BUCKET, story_file_path)
    for story_part in story_dialogue:
        image_file = story_dir / story_name / f"{story_name}_{story_part}.png"
        assert image_file.exists()
        upload_story_image(image_file, story_part, story_name)

## Upload challenges

In [ ]:
from src.chat import get_html_challenge_inputs, create_html_challenges
from src.gcs_storage import upload_to_gcs, get_story_challenges_path, get_story_translated_challenges_path, read_from_gcs
notebook_dir = Path().absolute()  # This gives src/notebooks
phrase_dir = notebook_dir.parent / "data" / "phrases" #where we store text files of phrases

for story_name in all_stories:
    # story_dir = notebook_dir.parent / "outputs" / "stories" / story_name
    # chat_dir = story_dir / config.TARGET_LANGUAGE_NAME
    # scenario_dicts = load_json(file_path=f"{story_dir}/scenarios.json")
    # upload the scenarios to gcs
    challenge_file_path = get_story_challenges_path(story_name, collection="LM1000")
    scenario_dicts = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=challenge_file_path)
    challenges = get_html_challenge_inputs(scenario_dicts)
    chat_webpage_file = create_html_challenges(challenges, story_name=story_name) # this creates and uploades
